In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11880,2024-05-26,Nova Zelândia Nbl,03:30,Franklin Bulls,Nelson Giants,1.20,4.17,175.5,1.85,1.85,0.0,0.00,0.00,vBZSYtwJ,0.833333,0.239808,0.540541,0.540541,0.073141,135.354,36.030400,0.266194,1.8,1.643168,0.912871,143.65,1.526,0.545921,0.357747,51.0,256.476,192.577506,0.750860,1.8,1.643168,0.912871,573.30,2.604,1.775241,0.681736,2.0,85,105,1.69,5.46,122.342,0.000,0.782163,0.000000,NaN,-0.34,-0.068,-2.941176,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11881,2024-05-26,Nova Zelândia Nbl,00:00,Manawatu Jets,Whai,1.98,1.75,165.5,1.85,1.81,1.5,1.82,1.63,0KzPZMNC,0.505051,0.571429,0.540541,0.552486,0.076479,698.098,229.696942,0.329033,0.0,0.000000,NaN,959.00,9.284,3.393307,0.365501,-135.0,431.404,199.705862,0.462921,0.6,1.341641,2.236068,457.92,5.930,2.706381,0.456388,-70.0,70,72,13.70,6.36,0.000,0.000,0.087204,0.015456,0.077884,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11882,2024-05-26,Lituânia Lkl,11:20,Zalgiris Kaunas,Lietkabelis,1.10,6.37,162.5,1.87,1.87,0.0,0.00,0.00,bcBvE0N9,0.909091,0.156986,0.534759,0.534759,0.066077,107.836,9.556028,0.088616,2.4,1.341641,0.559017,110.70,1.150,0.123693,0.107559,72.0,257.632,126.552544,0.491214,1.8,1.643168,0.912871,353.33,3.102,1.843670,0.594349,-6.0,90,89,1.23,3.97,103.708,219.558,0.997712,0.000000,NaN,0.48,0.096,1.041667,0.777615,0.9,0.122385,0.59,0.118,45.508475,0.511249,0.6,0.088751
11883,2024-05-26,Japão B.League,01:10,Ryukyu,Hiroshima D.,1.49,2.50,144.5,1.83,1.85,0.0,0.00,0.00,dOoun9ht,0.671141,0.400000,0.546448,0.540541,0.071141,142.750,38.288074,0.268218,1.8,1.643168,0.912871,105.08,2.084,0.587137,0.281736,-8.0,197.562,17.509491,0.088628,2.4,1.341641,0.559017,168.64,2.706,0.167422,0.061871,8.0,74,62,1.42,2.72,123.816,197.562,0.357984,0.007686,NaN,0.92,0.184,2.663043,0.753659,0.8,0.046341,5.81,1.162,1.290878,0.452582,0.8,0.347418
11884,2024-05-26,Itália Série A2,14:00,Udine,Cantu,1.77,1.95,146.5,1.87,1.79,-1.5,1.91,2.12,pKzDztxH,0.564972,0.512821,0.534759,0.558659,0.077792,127.214,30.814866,0.242229,2.4,1.341641,0.559017,129.20,1.638,0.640718,0.391159,62.0,111.536,8.273263,0.074176,1.8,1.643168,0.912871,124.20,1.578,0.303513,0.192340,6.0,76,60,1.70,2.07,107.364,124.404,0.068430,0.030912,0.073694,0.34,0.068,11.323529,0.770409,0.8,0.029591,-0.97,-0.194,-4.896907,0.670835,0.5,-0.170835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11942,2024-05-26,Austrália Nbl1 Central,00:00,Norwood Flames F,Central Districts Lions F,3.77,1.24,148.5,1.85,1.85,0.0,0.00,0.00,rT3L4iDU,0.265252,0.806452,0.540541,0.540541,0.071704,175.028,51.650366,0.295098,1.8,1.643168,0.912871,153.36,2.378,0.898899,0.378006,82.0,494.312,806.088602,1.630728,2.4,1.341641,0.559017,1936.00,5.622,9.156474,1.628686,56.0,54,88,2.84,22.00,0.000,0.000,0.714164,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11943,2024-05-26,Austrália Nbl1 Central,03:45,Southern Tigers,Woodville Warriors,3.06,1.34,164.5,1.85,1.83,0.0,0.00,0.00,bLqdYizm,0.326797,0.746269,0.540541,0.546448,0.073066,300.896,154.336291,0.512922,1.2,1.643168,1.369306,202.34,4.07

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11908,19:00,Porto Rico Bsn,San German,Indios de Mayaguez,1.72,Back Home
11910,10:30,Rússia Liga Vtb United,Unics Kazan,CSKA Moscow,1.69,Back Home
11912,00:00,Austrália Nbl1 Leste,Newcastle Falcons F,Norths Bears F,1.55,Back Home
11914,19:00,Porto Rico Bsn,Vaqueros de Bayamon,Leones De Ponce,1.55,Back Home
11925,02:15,Austrália Nbl1 Leste,Newcastle Falcons,Norths Bears,2.22,Back Home
11934,12:00,Espanha Leb Ouro,Gipuzkoa,Tizona Burgos,1.80,Back Home
